In [205]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math
import copy

#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l1

In [206]:

fashion_mnist = keras.datasets.fashion_mnist

(train_data, train_label), (test_data, test_label) = fashion_mnist.load_data()

In [207]:
norm_train_data = train_data/255
norm_test_data = test_data/255

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


In [208]:
def flat_2d_data(data):
    flatten_data=[]
    for i in range(data.shape[0]):
        flatten_data.append([])
        flatten_data[i] = data[i].flatten() 
    return np.asarray(flatten_data)

norm_train_data = train_data/255
norm_test_data = test_data/255

norm_train_data = flat_2d_data(norm_train_data)
norm_test_data = flat_2d_data(norm_test_data)

In [209]:
def my_optimizer():
    return keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

def build_model_Q4(input_size):
    model = keras.Sequential()
    model.add(layers.Dense(128, activation='relu',input_dim = input_size, activity_regularizer=l1(0.001)))
    model.add(layers.Dense(64, activation='relu', activity_regularizer=l1(0.001)))
    model.add(layers.Dense(10))
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer = my_optimizer(),  metrics=['accuracy'])
    return model

def step_decay(epoch):
    initial_lrate = 0.002
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate


batchsize = 32
num_epochs = 100
lrate = keras.callbacks.LearningRateScheduler(step_decay, verbose=0)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [210]:
def build_auto_encoder(out_dimension):
    model = keras.Sequential()
    model.add(layers.Dense(4*out_dimension, activation='relu',input_dim = 784))
    model.add(layers.Dense(2*out_dimension, activation='relu'))
    model.add(layers.Dense(out_dimension, activation='relu',name="out_encoders"))
    model.add(layers.Dense(2*out_dimension, activation='relu'))
    model.add(layers.Dense(4*out_dimension, activation='relu'))
    model.add(layers.Dense(784, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer = "adam")
    return model

In [211]:
auto_encoder_model = build_auto_encoder(84)
auto_encoder_model.summary()

Model: "sequential_53"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_128 (Dense)           (None, 336)               263760    
                                                                 
 dense_129 (Dense)           (None, 168)               56616     
                                                                 
 out_encoders (Dense)        (None, 84)                14196     
                                                                 
 dense_130 (Dense)           (None, 168)               14280     
                                                                 
 dense_131 (Dense)           (None, 336)               56784     
                                                                 
 dense_132 (Dense)           (None, 784)               264208    
                                                                 
Total params: 669,844
Trainable params: 669,844
Non-t

In [212]:
history = auto_encoder_model.fit(norm_train_data, norm_train_data,
                    epochs=10,
                    batch_size=20,
                    validation_split = 0.2,
                    callbacks=[early_stop, lrate])

Epoch 1/10
2400/2400 [==============================] - 13s 5ms/step - loss: 0.3044 - val_loss: 0.2900 - lr: 0.0020
Epoch 2/10
2400/2400 [==============================] - 14s 6ms/step - loss: 0.2859 - val_loss: 0.2854 - lr: 0.0020
Epoch 3/10
2400/2400 [==============================] - 14s 6ms/step - loss: 0.2822 - val_loss: 0.2833 - lr: 0.0020
Epoch 4/10
2400/2400 [==============================] - 13s 6ms/step - loss: 0.2805 - val_loss: 0.2812 - lr: 0.0020
Epoch 5/10
2400/2400 [==============================] - 14s 6ms/step - loss: 0.2793 - val_loss: 0.2812 - lr: 0.0020
Epoch 6/10
2400/2400 [==============================] - 14s 6ms/step - loss: 0.2784 - val_loss: 0.2794 - lr: 0.0020
Epoch 7/10
2400/2400 [==============================] - 14s 6ms/step - loss: 0.2776 - val_loss: 0.2797 - lr: 0.0020
Epoch 8/10
2400/2400 [==============================] - 14s 6ms/step - loss: 0.2770 - val_loss: 0.2793 - lr: 0.0020
Epoch 9/10
2400/2400 [==============================] - 14s 6ms/step - l

In [213]:
out_auto_encoder = auto_encoder_model.predict(norm_train_data)


1875/1875 [==============================] - 5s 3ms/step


In [214]:
layer_output = auto_encoder_model.get_layer("out_encoders").output
intermediate_model = keras.models.Model(inputs = auto_encoder_model.input,outputs=layer_output)

In [215]:
encoded_train_data = intermediate_model.predict(norm_train_data)
encoded_test_data = intermediate_model.predict(norm_test_data)

313/313 [==============================] - 0s 1ms/step


In [216]:
model_auto_encoder = build_model_Q4(84)
model_auto_encoder.summary()
history = model_auto_encoder.fit(encoded_train_data, train_label,
                    epochs=num_epochs,
                    batch_size=batchsize,
                    validation_split = 0.2,
                    # validation_data=(norm_test_data, test_label),
                    callbacks=[early_stop, lrate])

Model: "sequential_54"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_133 (Dense)           (None, 128)               10880     
                                                                 
 dense_134 (Dense)           (None, 64)                8256      
                                                                 
 dense_135 (Dense)           (None, 10)                650       
                                                                 
Total params: 19,786
Trainable params: 19,786
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5480 - accuracy: 0.8246 - val_loss: 0.4489 - val_accuracy: 0.8529 - lr: 0.0020
Epoch 2/100
1500/1500 [==============================] - 2s 2ms/step - loss: 0.4184 - accuracy: 0.8601 - val_loss: 0.4407 - val_accuracy: 0.8487 - lr: 0.00

In [217]:
test_loss, test_acc = model_auto_encoder.evaluate(encoded_test_data,  test_label, verbose=2)

print('\nTest accuracy:', test_acc)
print('\nTest loss:', test_loss)

313/313 - 0s - loss: 0.3953 - accuracy: 0.8696 - 401ms/epoch - 1ms/step

Test accuracy: 0.8695999979972839

Test loss: 0.3953070044517517


In [218]:
##########RBM

In [219]:
class LayerTranspose(keras.layers.Layer):
    def __init__(self, dense, activation, **kwargs):
        self.dense = dense
        self.activation = keras.activations.get(activation)
        super().__init__(**kwargs)
  
    def build(self, batch_input_shape):
        self.biases = self.add_weight(name = "bias",
                                      shape = [self.dense.input_shape[-1]],
                                      initializer = "zeros")
        super().build(batch_input_shape)
  
    def call(self, inputs):
        z = tf.matmul(inputs, self.dense.weights[0], transpose_b = True)
        return self.activation(z + self.biases)

In [220]:
def build_RBM_encoder(out_dimension):
    model = keras.Sequential()
  
    layer0 = layers.Flatten(input_shape=(28*28,1))
    layer1 = layers.Dense(784, activation='relu')
    layer2 = layers.Dense(4*out_dimension, activation='relu')
    layer3 = layers.Dense(2*out_dimension, activation='relu')
    layer4 = layers.Dense(out_dimension, activation='relu',name="out_encoders")

    model.add(layer0)
    model.add(layer1)
    model.add(layer2)
    model.add(layer3)
    model.add(layer4)

    model.add(LayerTranspose(layer4, activation='relu'))
    model.add(LayerTranspose(layer3, activation='relu'))
    model.add(LayerTranspose(layer2, activation='relu'))
    model.add(LayerTranspose(layer1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer = "adam")
    return model

In [221]:
RBM_encoder_model = build_RBM_encoder(84)
RBM_encoder_model.summary()

Model: "sequential_55"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_34 (Flatten)        (None, 784)               0         
                                                                 
 dense_136 (Dense)           (None, 784)               615440    
                                                                 
 dense_137 (Dense)           (None, 336)               263760    
                                                                 
 dense_138 (Dense)           (None, 168)               56616     
                                                                 
 out_encoders (Dense)        (None, 84)                14196     
                                                                 
 layer_transpose_24 (LayerTr  (None, 168)              14364     
 anspose)                                                        
                                                     

In [222]:
history = RBM_encoder_model.fit(norm_train_data, norm_train_data,
                    epochs=15,
                    batch_size=256,
                    validation_split = 0.2,
                    callbacks=[early_stop, lrate])

Epoch 1/15
188/188 [==============================] - 8s 40ms/step - loss: 0.3420 - val_loss: 0.3042 - lr: 0.0020
Epoch 2/15
188/188 [==============================] - 7s 36ms/step - loss: 0.2941 - val_loss: 0.2891 - lr: 0.0020
Epoch 3/15
188/188 [==============================] - 6s 34ms/step - loss: 0.2854 - val_loss: 0.2837 - lr: 0.0020
Epoch 4/15
188/188 [==============================] - 7s 36ms/step - loss: 0.2799 - val_loss: 0.2799 - lr: 0.0020
Epoch 5/15
188/188 [==============================] - 7s 35ms/step - loss: 0.2769 - val_loss: 0.2773 - lr: 0.0020
Epoch 6/15
188/188 [==============================] - 7s 36ms/step - loss: 0.2745 - val_loss: 0.2757 - lr: 0.0020
Epoch 7/15
188/188 [==============================] - 6s 33ms/step - loss: 0.2730 - val_loss: 0.2740 - lr: 0.0020
Epoch 8/15
188/188 [==============================] - 7s 38ms/step - loss: 0.2715 - val_loss: 0.2735 - lr: 0.0020
Epoch 9/15
188/188 [==============================] - 7s 39ms/step - loss: 0.2704 - val_

In [223]:
layer_output = RBM_encoder_model.get_layer("out_encoders").output
intermediate_model = keras.models.Model(inputs = RBM_encoder_model.input,outputs=layer_output)

In [224]:
RBM_encoded_train_data = intermediate_model.predict(norm_train_data)
RBM_encoded_test_data = intermediate_model.predict(norm_test_data)

313/313 [==============================] - 1s 3ms/step


In [225]:
model_RBM_encoder = build_model_Q4(84)
model_RBM_encoder.summary()
history = model_RBM_encoder.fit(RBM_encoded_train_data, train_label,
                    epochs=num_epochs,
                    batch_size=batchsize,
                    validation_split = 0.2,
                    # validation_data=(norm_test_data, test_label),
                    callbacks=[early_stop, lrate])

Model: "sequential_56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_139 (Dense)           (None, 128)               10880     
                                                                 
 dense_140 (Dense)           (None, 64)                8256      
                                                                 
 dense_141 (Dense)           (None, 10)                650       
                                                                 
Total params: 19,786
Trainable params: 19,786
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5516 - accuracy: 0.8246 - val_loss: 0.4462 - val_accuracy: 0.8529 - lr: 0.0020
Epoch 2/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.4182 - accuracy: 0.8617 - val_loss: 0.4389 - val_accuracy: 0.8512 - lr: 0.00

In [226]:
test_loss, test_acc = model_RBM_encoder.evaluate(RBM_encoded_test_data,  test_label, verbose=2)

print('\nTest accuracy:', test_acc)
print('\nTest loss:', test_loss)

313/313 - 1s - loss: 0.3702 - accuracy: 0.8831 - 524ms/epoch - 2ms/step

Test accuracy: 0.8830999732017517

Test loss: 0.37016016244888306


In [227]:
#########3MLP

In [233]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import Constant
from keras.layers import Input, Dense


In [243]:
def my_optimizer():
    return keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

def build_model():
   # model_RBM_encoder
    model = keras.Sequential()
    model_RBM_encoder,
    model.add(layers.Flatten(input_shape=(28*28, 1)))
    model.add(layers.Dense(128, activation='relu', activity_regularizer=l1(0.001)))
    model.add(layers.Dense(64, activation='relu', activity_regularizer=l1(0.001)))
    model.add(layers.Dense(10))
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer = my_optimizer(),  metrics=['accuracy'])
    return model

def step_decay(epoch):
    initial_lrate = 0.002
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

In [244]:
batchsize = 256
num_epochs = 100


lrate = keras.callbacks.LearningRateScheduler(step_decay, verbose=0)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
myhistory = model.fit(norm_train_data, train_label,
                    epochs=num_epochs,
                    batch_size=batchsize,
                    validation_split = 0.2,
                    # validation_data=(norm_test_data, test_label),
                    callbacks=[early_stop, lrate])

Epoch 1/100
188/188 [==============================] - 1s 4ms/step - loss: 0.2580 - accuracy: 0.9150 - val_loss: 0.3943 - val_accuracy: 0.8733 - lr: 0.0020
Epoch 2/100
188/188 [==============================] - 1s 4ms/step - loss: 0.2416 - accuracy: 0.9200 - val_loss: 0.3945 - val_accuracy: 0.8739 - lr: 0.0020
Epoch 3/100
188/188 [==============================] - 1s 3ms/step - loss: 0.2173 - accuracy: 0.9285 - val_loss: 0.3590 - val_accuracy: 0.8911 - lr: 0.0020
Epoch 4/100
188/188 [==============================] - 1s 3ms/step - loss: 0.2109 - accuracy: 0.9316 - val_loss: 0.3706 - val_accuracy: 0.8862 - lr: 0.0020
Epoch 5/100
188/188 [==============================] - 1s 4ms/step - loss: 0.2097 - accuracy: 0.9315 - val_loss: 0.3566 - val_accuracy: 0.8917 - lr: 0.0020
Epoch 6/100
188/188 [==============================] - 1s 3ms/step - loss: 0.1999 - accuracy: 0.9355 - val_loss: 0.3842 - val_accuracy: 0.8813 - lr: 0.0020
Epoch 7/100
188/188 [==============================] - 1s 4ms/st

In [245]:
test_loss, test_acc = model.evaluate(norm_test_data,  test_label, verbose=2)

print('\nTest accuracy:', test_acc)
print('\nTest loss:', test_loss)

313/313 - 0s - loss: 0.4280 - accuracy: 0.8906 - 344ms/epoch - 1ms/step

Test accuracy: 0.8906000256538391

Test loss: 0.4280080199241638
